In [6]:
import pandas as pd

df_full = pd.read_csv('./data/ad_placement/census-income.csv')
                 
df_full.columns = ['AAGE','ACLSWKR','ADTIND','ADTOCC','AHGA','AHRSPAY','AHSCOL','AMARITL',
                                                                              'AMJIND','AMJOCC','ARACE','AREORGN','ASEX','AUNMEM','AUNTYPE','AWKSTAT','CAPGAIN',
                                                                              'CAPLOSS','DIVVAL','FILESTAT','GRINREG','GRINST','HHDFMX','HHDREL', 'MARSUPWT',
                                                                              'MIGMTR1','MIGMTR3','MIGMTR4','MIGSAME','MIGSUN','NOEMP','PARENT','PEFNTVTY',
                                                                              'PEMNTVTY','PENATVTY','PRCITSHP','SEOTR','VETQVA','VETYN','WKSWORK', 'YEAR', 'INCOME']

df_full.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,YEAR,INCOME
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,...,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,- 50000.


In [7]:
df_reduced=df_full[['AAGE','ACLSWKR','ADTIND','ADTOCC','AHGA','AHRSPAY','AMARITL','ARACE','ASEX','AUNTYPE','AWKSTAT','CAPGAIN',
          'CAPLOSS','DIVVAL','FILESTAT','HHDREL','MIGMTR1','NOEMP','PARENT','SEOTR','VETYN','WKSWORK', 'YEAR', 'INCOME']]

df_reduced.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AMARITL,ARACE,ASEX,AUNTYPE,...,FILESTAT,HHDREL,MIGMTR1,NOEMP,PARENT,SEOTR,VETYN,WKSWORK,YEAR,INCOME
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Divorced,White,Male,Not in universe,...,Head of household,Householder,MSA to MSA,1,Not in universe,0,2,52,94,- 50000.
1,18,Not in universe,0,0,10th grade,0,Never married,Asian or Pacific Islander,Female,Not in universe,...,Nonfiler,Child 18 or older,?,0,Not in universe,0,2,0,95,- 50000.
2,9,Not in universe,0,0,Children,0,Never married,White,Female,Not in universe,...,Nonfiler,Child under 18 never married,Nonmover,0,Both parents present,0,0,0,94,- 50000.
3,10,Not in universe,0,0,Children,0,Never married,White,Female,Not in universe,...,Nonfiler,Child under 18 never married,Nonmover,0,Both parents present,0,0,0,94,- 50000.
4,48,Private,40,10,Some college but no degree,1200,Married-civilian spouse present,Amer Indian Aleut or Eskimo,Female,Not in universe,...,Joint both under 65,Spouse of householder,?,1,Not in universe,2,2,52,95,- 50000.


In [11]:
df.dtypes

AAGE          int64
ACLSWKR      object
ADTIND        int64
ADTOCC        int64
AHGA         object
AHRSPAY       int64
AHSCOL       object
AMARITL      object
AMJIND       object
AMJOCC       object
ARACE        object
AREORGN      object
ASEX         object
AUNMEM       object
AUNTYPE      object
AWKSTAT      object
CAPGAIN       int64
CAPLOSS       int64
DIVVAL        int64
FILESTAT     object
GRINREG      object
GRINST       object
HHDFMX       object
HHDREL       object
MARSUPWT    float64
MIGMTR1      object
MIGMTR3      object
MIGMTR4      object
MIGSAME      object
MIGSUN       object
NOEMP         int64
PARENT       object
PEFNTVTY     object
PEMNTVTY     object
PENATVTY     object
PRCITSHP     object
SEOTR         int64
VETQVA       object
VETYN         int64
WKSWORK       int64
YEAR          int64
INCOME       object
dtype: object

In [11]:
df_dummies=pd.get_dummies(df_reduced)
df_continuous=df_reduced[['AAGE','ADTIND','ADTOCC','AHRSPAY','CAPGAIN','CAPLOSS','DIVVAL','NOEMP','SEOTR','VETYN','WKSWORK', 'YEAR']]

In [16]:
print df_full.shape
print df_reduced.shape
print df_dummies.shape
print df_continuous.shape

(199522, 42)
(199522, 24)
(199522, 97)
(199522, 12)


In [23]:
from sklearn import cluster
from sklearn import preprocessing

df_scaled=preprocessing.scale(df_continuous)

k_means = cluster.KMeans(n_clusters=5)
k_means.fit(df_scaled) 

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [24]:
clusters=k_means.predict(df_scaled)

In [27]:
clusters

array([0, 2, 1, ..., 0, 2, 0])

In [28]:
print(k_means.labels_[::10])

[0 2 0 ..., 1 0 2]
